<a href="https://colab.research.google.com/github/Gustavo-Ros/Simulacion-1/blob/main/Ejercicio_CossBu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [171]:
import random as rnd
import numpy as np
from tabulate import tabulate
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import minimize_scalar

In [82]:
def demanda_1():                            #Esta función calcula y devuelve la demanda
  prob=[0,0.05,0.1,0.2,0.35,0.6,0.85,1]
  u=rnd.random()
  for i in range(len(prob)):
      if prob[i-1] <= u <= prob[i]:       #Ubica el valor de u en inervalos
          x=i+4                           #Asigna valor de demanda
          break
  return x

In [83]:
def demanda_2():                            #Esta función calcula y devuelve la demanda
  prob=[0,0.15,0.35,0.65,0.85,1]
  u=rnd.random()
  for i in range(len(prob)):
      if prob[i-1] <= u <= prob[i]:       #Ubica el valor de u en inervalos
          x=i+3                           #Asigna valor de demanda
          break
  return x

In [164]:
def ganancias(ganancia,demanda,revistas,gastos,precio_venta):    #  Calcula las ganancias del periodo en curso

  if demanda > revistas:
    ventas=revistas*precio_venta
  else:
    ventas=demanda*precio_venta

  ganancia+=ventas-gastos
  return ganancia

In [89]:
costo_revistas1=1.5
sobrantes_final=0.6
costo2=1.2
precio_sobrantes=0.9
precio_venta=2

sobrantes=rnd.randint(1,10)

In [ ]:
compra1=rnd.randint(1,19)

In [166]:
def PERIODO_1(compra1,costo_revistas1,precio_venta):
  tabla=[]
  gastos=compra1*costo        # Calcula los gastos
  ganancia_total=0
  demanda1=demanda_1()        # Calcula la demanda en el primer periodo

  if compra1-demanda1 <= 0:
    inventario=0
  else:
    inventario=compra1-demanda1

  # Calcula las ventas y ganancias de las ventas en el primer periodo
  ganancia_total=ganancias(ganancia_total,demanda1,compra1,gastos,precio_venta)
  tabla.append(["Periodo 1",f"{compra1}",f"{demanda1}",f"{gastos}",f"{ganancia_total}",f"{inventario}"])
  return inventario,ganancia_total,tabla

In [193]:
#-------------------------------  PERIODO 1 : Primeros 10 días------------------
compra1=rnd.randint(1,19)
inventario,ganancia_total,tabla=PERIODO_1(compra1,costo_revistas1,precio_venta)

headers=["","Revistas compradas","demanda","Gastos","Ganancia","Inventario"]
print(tabulate(tabla, headers=headers, tablefmt="fancy_grid",showindex=False))

╒═══════════╤══════════════════════╤═══════════╤══════════╤════════════╤══════════════╕
│           │   Revistas compradas │   demanda │   Gastos │   Ganancia │   Inventario │
╞═══════════╪══════════════════════╪═══════════╪══════════╪════════════╪══════════════╡
│ Periodo 1 │                   19 │        10 │     28.5 │       -8.5 │            9 │
╘═══════════╧══════════════════════╧═══════════╧══════════╧════════════╧══════════════╛


In [199]:
compra2=rnd.randint(0,8)
#sobrantes=rnd.randint(1,10)
demanda2=demanda_2()        # Calcula la demanda en el segundo periodo
if inventario <= 4:
  sobrantes=0


In [198]:
demanda2

7

In [197]:
def restriccion(sobrantes):
    return inventario - sobrantes  # sobrantes debe ser <= inventari

resultado = minimize_scalar(lambda x: ganancias(ganancia_total,demanda2,inventario-sobrantes,0,precio_sobrantes), bounds=(0, inventario), method='bounded')
sobrantes_optimizado = np.round(resultado.x)
# Mostrar resultados
print(f"Sobrantes óptimo para maximizar ganancias: {sobrantes_optimizado}")
print(f"Ganancias máximas obtenidas: {ganancias(ganancia_total,demanda2,inventario-sobrantes_optimizado,0,precio_sobrantes)}")

Sobrantes óptimo para maximizar ganancias: 9.0
Ganancias máximas obtenidas: -8.5
